<a href="https://colab.research.google.com/github/dukes101/LOM-Analytics/blob/main/MatchupAnalytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analytics

In [75]:
## Connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [76]:
## Set libraries and read in data
import pandas as pd
import numpy as np

# Read in matchup data from MatchupDataPull.ipynb
dfMatchupsPd = pd.read_csv('/content/drive/My Drive/dfAllMatchups.csv')

## Head to Head

The head to head data frame (dfH2HFinal) is a summary of every possible matchup in the league

### H2H Data Clean

In [77]:
## Define member list
league_members = ['Luca Hurst', 'Dylan Peters', 'Cody Morton', 'Matt Willadsen', 'Nate Stone', 'Hayden Bingham', 'Gavin Tacto', 'Zachary Toma', 'Alexander Nicoll', 'Evan Williams']

In [78]:
## Create a new column to store the winner of each matchup (or a tie)
dfMatchupsPd['Winner'] = dfMatchupsPd.apply(lambda row: row['Team1'] \
                                            if row['Score1'] > row['Score2'] \
                                            else (row['Team2'] if row['Score1'] < row['Score2'] else 'Tie'), axis=1)

In [79]:
## Function to swap teams
## Identifies h2h winner and loser
def swap_teams(df, member1, member2):

    ## Calculate win counts for each member
    member1_wins = (df['Winner'] == member1).sum()
    member2_wins = (df['Winner'] == member2).sum()

    winning_member = member1 if member1_wins >= member2_wins else member2
    losing_member = member2 if member1_wins >= member2_wins else member1

    ## Iterate through each matchup
    for index, row in df.iterrows():

      # Forcing the winning_member to be 'Team1'
      if row['Team1'] != winning_member:
          # Swap team and score columns for the current row
          df.loc[index, ['Team1', 'Team2']] = row[['Team2', 'Team1']].values
          df.loc[index, ['Score1', 'Score2']] = row[['Score2', 'Score1']].values

    return df, winning_member, losing_member

In [80]:
## Function to calculate h2h metrics
def h2h_metrics(df, winning_member, losing_member):

  # Score margins
  avg_score_margin = round((df['Score1'] - df['Score2']).mean(),1) #team 1 margin
  avg_score_margin_abs = round((df['Score1'] - df['Score2']).abs().mean(),1) #winning team margin

  # Total matchups
  total_matchups = round(len(df),0)

  # Ties
  tie_games = round((df['Winner'] == 'Tie').sum(),0)

  # Team names
  team1 = df['Team1'].iloc[0]
  team2 = df['Team2'].iloc[0]

  # Total points
  team1_points = round(df['Score1'].sum(),1)
  team2_points = round(df['Score2'].sum(),1)
  allteam_points = team1_points + team2_points

  # Total wins
  team1_wins = round((df['Winner'] == winning_member).sum(),0)
  team2_wins = round((df['Winner'] == losing_member).sum(),0)

  # Win pct
  team1_win_pct = round((team1_wins / (total_matchups - tie_games)) * 100,1)
  team2_win_pct = round((team2_wins / (total_matchups - tie_games)) * 100,1)

  # Playoff games
  playoff_games = round((df['playoffType'] != 'NONE').sum(),0)

  # Df of results
  results_df = pd.DataFrame({
      'Team 1': [team1],
      'Team 2': [team2],
      'Team 1 Points': [team1_points],
      'Team 2 Points': [team2_points],
      'Total H2H Points': [allteam_points],
      'Average Score Margin (Team 1)': [avg_score_margin],
      'Average Score Margin (Winning Team)': [avg_score_margin_abs],
      'Total Matchups': [total_matchups],
      'Team 1 Wins': [team1_wins],
      'Team 2 Wins': [team2_wins],
      'Team 1 Win %': [team1_win_pct],
      'Team 2 Win %': [team2_win_pct],
      'Tie Games': [tie_games],
      'Playoff Games': [playoff_games]
  })

  return results_df

In [81]:
## Initialize dfH2HAll
data = {
    'Team 1': [],
    'Team 2': [],
    'Team 1 Points': [],
    'Team 2 Points': [],
    'Total H2H Points': [],
    'Average Score Margin (Team 1)': [],
    'Average Score Margin (Winning Team)': [],
    'Total Matchups': [],
    'Team 1 Wins': [],
    'Team 2 Wins': [],
    'Team 1 Win %': [],
    'Team 2 Win %': [],
    'Tie Games': [],
    'Playoff Games': []
}

dfH2HAll = pd.DataFrame(data)

In [82]:
## Build loop for all head to head matchups
for member1 in league_members:
    for member2 in league_members:

      ## Ensure we aren't comparing the same members
      if member1 != member2:

        ## Filter on current iteration of member matchups
        dfH2HRaw = dfMatchupsPd[((dfMatchupsPd['Team1'] == member1) & (dfMatchupsPd['Team2'] == member2)) \
                                | ((dfMatchupsPd['Team1'] == member2) & (dfMatchupsPd['Team2'] == member1))]

        ## Swap teams if not aligned in h2h
        dfH2HSwap, winning_member, losing_member = swap_teams(dfH2HRaw, member1, member2)
        #dfH2HSwap = dfH2HRaw.apply(swap_teams, axis=1)

        ## Calculate h2h metrics
        dfH2H = h2h_metrics(dfH2HSwap, winning_member, losing_member)

      ## Continue to next iteration if they are the same member
      else:
        continue

      ## Append all h2h history
      dfH2HAll = pd.concat([dfH2HAll, dfH2H], ignore_index=True)

dfH2HAll['key'] = dfH2HAll.apply(lambda row: tuple(sorted([row['Team 1'], row['Team 2']])), axis=1)
dfH2HFinal = dfH2HAll.drop_duplicates(subset='key', keep='first').drop('key', axis=1).reset_index(drop=True)

### H2H Display

In [83]:
dfH2HFinal.head()

,Team 1,Team 2,Team 1 Points,Team 2 Points,Total H2H Points,Average Score Margin (Team 1),Average Score Margin (Winning Team),Total Matchups,Team 1 Wins,Team 2 Wins,Team 1 Win %,Team 2 Win %,Tie Games,Playoff Games
0,Luca Hurst,Dylan Peters,1078.0,966.2,2044.2,16.0,28.3,7.0,5.0,2.0,71.4,28.6,0.0,1.0
1,Luca Hurst,Cody Morton,2306.0,2111.8,4417.8,12.1,25.5,16.0,9.0,7.0,56.2,43.8,0.0,1.0
2,Luca Hurst,Matt Willadsen,1949.3,1907.4,3856.7,3.0,30.2,14.0,8.0,6.0,57.1,42.9,0.0,3.0
3,Luca Hurst,Nate Stone,1889.6,1724.8,3614.4,12.7,26.1,13.0,8.0,5.0,61.5,38.5,0.0,1.0
4,Luca Hurst,Hayden Bingham,1578.7,1441.9,3020.6,13.7,33.5,10.0,7.0,3.0,70.0,30.0,0.0,2.0


### Easiest/Toughest Opponent

In [84]:
## Initialize dfversus
data = {
    'Team': [],
    'Easiest Opponent': [],
    'Easy Win % Against': [],
    'Toughest Opponent': [],
    'Tough Win % Against': [],
}

dfVersus = pd.DataFrame(data)

In [85]:
for team in league_members:

  df = dfH2HFinal[((dfH2HFinal['Team 1'] == team) | (dfH2HFinal['Team 2'] == team))]

  ## Iterate through each matchup
  for index, row in df.iterrows():

    # Forcing current team to be 'Team 1'
    if row['Team 1'] != team:
        # Swap team and score columns for the current row
        df.loc[index, ['Team 1', 'Team 2']] = row[['Team 2', 'Team 1']].values
        df.loc[index, ['Team 1 Win %', 'Team 2 Win %']] = row[['Team 2 Win %', 'Team 1 Win %']].values

  easiest_opponent = df.loc[df['Team 1 Win %'].idxmax()]['Team 2']
  easiest_opponent_winpct = df.loc[df['Team 1 Win %'].idxmax()]['Team 1 Win %']
  toughest_opponent = df.loc[df['Team 1 Win %'].idxmin()]['Team 2']
  toughest_opponent_winpct = df.loc[df['Team 1 Win %'].idxmin()]['Team 1 Win %']

  dfVersus = pd.concat([dfVersus, pd.DataFrame([{'Team': team
                              ,'Easiest Opponent': easiest_opponent
                              ,'Easy Win % Against': easiest_opponent_winpct
                              ,'Toughest Opponent': toughest_opponent
                              ,'Tough Win % Against': toughest_opponent_winpct}])], ignore_index=True)

In [86]:
dfVersus = dfVersus.sort_values(by='Easy Win % Against', ascending=False).reset_index(drop=True)

In [87]:
dfVersus.head()

,Team,Easiest Opponent,Easy Win % Against,Toughest Opponent,Tough Win % Against
0,Nate Stone,Cody Morton,83.3,Matt Willadsen,27.3
1,Cody Morton,Evan Williams,80.0,Nate Stone,16.7
2,Matt Willadsen,Nate Stone,72.7,Alexander Nicoll,37.5
3,Luca Hurst,Dylan Peters,71.4,Evan Williams,44.4
4,Gavin Tacto,Nate Stone,71.4,Cody Morton,40.0


## Hall of Fame

**Columns:** Year, Week, Team, Score, Opponent Score, Score Margin, Outcome(Win/Loss), Type(Regular/Playoff), Top Scoring Week (Yes/No), Low Scoring Week (Yes/No)

### Hall of Fame Data Clean

In [88]:
## Create columns for team history data set
melted_df = pd.melt(dfMatchupsPd, id_vars=['Week', 'Type', 'Year', 'Score1', 'Score2', 'Winner'],
                    value_vars=['Team1', 'Team2'],
                    var_name='team_type', value_name='Team')

melted_df['Outcome'] = np.where(melted_df['Team'] == melted_df['Winner'], 'Win', 'Loss')

melted_df['Score'] = np.where(melted_df['team_type'] == 'Team1', melted_df['Score1'], melted_df['Score2'])

melted_df['Opponent_score'] = np.where(melted_df['team_type'] == 'Team1', melted_df['Score2'], melted_df['Score1'])

melted_df['Score_margin'] = melted_df['Score'] - melted_df['Opponent_score']

melted_df = melted_df.drop(['Score1', 'Score2', 'Winner', 'team_type'], axis=1)

melted_df['Top_scoring_week'] = melted_df.groupby(['Year', 'Week'])['Score'].transform(lambda x: (x == x.max()).astype(int))

melted_df['Lowest_scoring_week'] = melted_df.groupby(['Year', 'Week'])['Score'].transform(lambda x: (x == x.min()).astype(int))

dfTeamHistory = melted_df[['Year', 'Week', 'Team', 'Score', 'Opponent_score', 'Score_margin', 'Outcome', 'Type', 'Top_scoring_week', 'Lowest_scoring_week']]

In [89]:
dfTeamHistory = dfTeamHistory.sort_values(by=['Year', 'Week', 'Score'], ascending=[True, True, False]).reset_index(drop=True)

In [90]:
dfTeamHistory.head()

,Year,Week,Team,Score,Opponent_score,Score_margin,Outcome,Type,Top_scoring_week,Lowest_scoring_week
0,2017,1.0,Dylan Peters,154.0,120.0,34.0,Win,Regular,1,0
1,2017,1.0,Cody Morton,120.0,154.0,-34.0,Loss,Regular,0,0
2,2017,1.0,Luca Hurst,111.0,96.0,15.0,Win,Regular,0,0
3,2017,1.0,Gavin Tacto,104.0,98.0,6.0,Win,Regular,0,0
4,2017,1.0,Hayden Bingham,103.0,99.0,4.0,Win,Regular,0,0


### Standings Summary

In [91]:
## Championship data frame
data = {
    'Year': [2017, 2018, 2019, 2020, 2021, 2022, 2023],
    'Champion': ['Dylan Peters', 'Luca Hurst', 'Gavin Tacto', 'Matt Willadsen', 'Gavin Tacto', 'Luca Hurst', 'Alexander Nicoll'],
    'Runner-Up': ['Luca Hurst', 'Matt Willadsen', 'Alexander Nicoll', 'Luca Hurst', 'Evan Williams', 'Cody Morton', 'Luca Hurst'],
    '3rd Place': ['Matt Willadsen', 'Gavin Tacto', 'Cody Morton', 'Hayden Bingham', 'Dylan Peters', 'Nate Stone', 'Hayden Bingham']
}

dfResults = pd.DataFrame(data)
dfResults.sort_values(by='Year', ascending=False).reset_index(drop=True)

,Year,Champion,Runner-Up,3rd Place
0,2023,Alexander Nicoll,Luca Hurst,Hayden Bingham
1,2022,Luca Hurst,Cody Morton,Nate Stone
2,2021,Gavin Tacto,Evan Williams,Dylan Peters
3,2020,Matt Willadsen,Luca Hurst,Hayden Bingham
4,2019,Gavin Tacto,Alexander Nicoll,Cody Morton
5,2018,Luca Hurst,Matt Willadsen,Gavin Tacto
6,2017,Dylan Peters,Luca Hurst,Matt Willadsen


In [92]:
## Results Summary
melted_results = pd.melt(dfResults, id_vars=['Year'], value_vars=['Champion', 'Runner-Up', '3rd Place'],
                    var_name='Finish', value_name='Team')

# Count championships and top 3 finishes
championships = melted_results[melted_results['Finish'] == 'Champion']['Team'].value_counts().reset_index()
championships.columns = ['Team', 'Championships']
top_3_finishes = melted_results['Team'].value_counts().reset_index()
top_3_finishes.columns = ['Team', 'Top 3 Finishes']

# Create output DataFrame
historic_output_df = pd.DataFrame({
    'Team': league_members,
    'Championships': [championships[championships['Team'] == team]['Championships'].values[0] if team in championships['Team'].values else 0 for team in league_members],
    'Top 3 Finishes': [top_3_finishes[top_3_finishes['Team'] == team]['Top 3 Finishes'].values[0] if team in top_3_finishes['Team'].values else 0 for team in league_members]
})

### All Time Team Performance
The following code will summarize the all time performance stats over the history of the league

In [93]:
## Total Points
dfPointsAllTime = dfTeamHistory.groupby('Team')['Score'].sum().sort_values(ascending=False).reset_index(name='Points')
dfPointsAllTime.head()

,Team,Points
0,Luca Hurst,14219.25
1,Gavin Tacto,13781.25
2,Matt Willadsen,13293.15
3,Dylan Peters,12666.85
4,Hayden Bingham,12407.15


In [94]:
## PPG (excluding playoffs)
dfPPGAllTime = dfTeamHistory[dfTeamHistory['Type'] != 'Playoff'].groupby('Team')['Score'].mean().sort_values(ascending=False).reset_index(name='PPG')
dfPPGAllTime.head()

,Team,PPG
0,Gavin Tacto,131.981111
1,Dylan Peters,131.015000
2,Luca Hurst,129.925556
3,Cody Morton,127.861111
4,Matt Willadsen,127.305556


In [95]:
## Total Playoff Games
dfPlayoffGamesAllTime = dfTeamHistory[dfTeamHistory['Type'] == 'Playoff'].groupby('Team')['Type'].count().sort_values(ascending=False).reset_index(name='Playoff Games')
dfPlayoffGamesAllTime.head()

,Team,Playoff Games
0,Luca Hurst,11
1,Gavin Tacto,9
2,Matt Willadsen,9
3,Hayden Bingham,6
4,Alexander Nicoll,5


In [96]:
## Total Wins (including playoffs)
dfWinsAllTime = dfTeamHistory[dfTeamHistory['Outcome'] == 'Win'].groupby('Team')['Outcome'].count().sort_values(ascending=False).reset_index(name='Wins')
dfWinsAllTime.head()

,Team,Wins
0,Luca Hurst,58
1,Matt Willadsen,56
2,Gavin Tacto,54
3,Dylan Peters,51
4,Hayden Bingham,45


In [97]:
## Win Percentage (including playoffs)
dfWinPctAllTime = dfTeamHistory.groupby('Team')['Outcome'].apply(lambda x: (x == 'Win').mean() * 100).sort_values(ascending=False).reset_index(name='Win %')
dfWinPctAllTime.head()

,Team,Win %
0,Luca Hurst,57.425743
1,Matt Willadsen,56.565657
2,Gavin Tacto,54.545455
3,Dylan Peters,54.255319
4,Alexander Nicoll,48.571429


In [98]:
## Win Percentage Against Each Team
dfWinPctAgainstOppAllTime = dfH2HFinal[['Team 1', 'Team 2', 'Team 1 Win %', 'Total Matchups']].sort_values(by='Team 1 Win %', ascending=False).reset_index(drop=True)
dfWinPctAgainstOppAllTime.head()

,Team 1,Team 2,Team 1 Win %,Total Matchups
0,Nate Stone,Cody Morton,83.3,12.0
1,Cody Morton,Evan Williams,80.0,5.0
2,Cody Morton,Zachary Toma,76.9,13.0
3,Matt Willadsen,Nate Stone,72.7,11.0
4,Luca Hurst,Dylan Peters,71.4,7.0


In [99]:
## Total Playoff Appearances
dfPlayoffAppAllTime = dfTeamHistory[dfTeamHistory['Type'] == 'Playoff'][['Year', 'Team', 'Type']].drop_duplicates().groupby('Team')['Type'].count().sort_values(ascending=False).reset_index(name='Playoff Appearances')
dfPlayoffAppAllTime.head()

,Team,Playoff Appearances
0,Gavin Tacto,6
1,Luca Hurst,5
2,Cody Morton,4
3,Hayden Bingham,4
4,Matt Willadsen,4


In [100]:
## Total Playoff Wins
dfPlayoffWinsAllTime = dfTeamHistory[dfTeamHistory['Type'] == 'Playoff'].groupby('Team')['Outcome'].apply(lambda x: (x == 'Win').sum()).sort_values(ascending=False).reset_index(name='Playoff Wins')
dfPlayoffWinsAllTime.head()

,Team,Playoff Wins
0,Luca Hurst,8
1,Matt Willadsen,6
2,Gavin Tacto,5
3,Alexander Nicoll,4
4,Dylan Peters,2


In [101]:
## Playoff Win Percentage
dfPlayoffWinPct = dfTeamHistory[dfTeamHistory['Type'] == 'Playoff'].groupby('Team')['Outcome'].apply(lambda x: (x == 'Win').mean() * 100).sort_values(ascending=False).reset_index(name='Playoff Win %')
dfPlayoffWinPct = pd.merge(dfPlayoffWinPct, dfPlayoffGamesAllTime, on='Team')
dfPlayoffWinPct.head()

,Team,Playoff Win %,Playoff Games
0,Alexander Nicoll,80.000000,5
1,Luca Hurst,72.727273,11
2,Matt Willadsen,66.666667,9
3,Gavin Tacto,55.555556,9
4,Dylan Peters,50.000000,4


In [102]:
## Top Scoring Weeks
dfTopScoringWeeksAllTime = dfTeamHistory[dfTeamHistory['Top_scoring_week'] == 1].groupby('Team')['Top_scoring_week'].count().sort_values(ascending=False).reset_index(name='Top Scoring Weeks')
dfTopScoringWeeksAllTime.head()

,Team,Top Scoring Weeks
0,Gavin Tacto,20
1,Luca Hurst,18
2,Matt Willadsen,15
3,Cody Morton,14
4,Dylan Peters,11


In [103]:
## Low Scoring Weeks
dfLowScoringWeeksAllTime = dfTeamHistory[dfTeamHistory['Lowest_scoring_week'] == 1].groupby('Team')['Lowest_scoring_week'].count().sort_values(ascending=False).reset_index(name='Low Scoring Weeks')
dfLowScoringWeeksAllTime.head()

,Team,Low Scoring Weeks
0,Zachary Toma,14
1,Matt Willadsen,13
2,Nate Stone,13
3,Cody Morton,12
4,Evan Williams,12


In [104]:
## Average Score Margin (excluding playoffs)
dfScoreMarginAllTime = dfTeamHistory[dfTeamHistory['Type'] != 'Playoff'].groupby('Team')['Score_margin'].mean().sort_values(ascending=False).reset_index(name='Avg Score Margin')
dfScoreMarginAllTime.head()

,Team,Avg Score Margin
0,Dylan Peters,6.062778
1,Gavin Tacto,5.568333
2,Matt Willadsen,3.499444
3,Luca Hurst,3.436111
4,Cody Morton,0.408889


In [105]:
## Performance Summary (Championships, Top 3 Finishes, Playoff Appearances)
historic_team_performance = pd.merge(historic_output_df, dfPlayoffAppAllTime, on='Team', how='left')
historic_team_performance.head()

,Team,Championships,Top 3 Finishes,Playoff Appearances
0,Luca Hurst,2,5,5
1,Dylan Peters,1,2,3
2,Cody Morton,0,2,4
3,Matt Willadsen,1,3,4
4,Nate Stone,0,1,4


### Year by Year Performance

**Calculate:** Points, PPG, Score Margin, Wins, Win %, Finishing Position, Roto Points

In [106]:
## Points per year
dfPointsYearbyYear = dfTeamHistory[dfTeamHistory['Type'] != 'Playoff'].groupby(['Team', 'Year'])['Score'].sum().reset_index(name='Points').sort_values(by=['Team', 'Year'], ascending=[True, True])
dfPointsYearbyYear.head()

,Team,Year,Points
0,Alexander Nicoll,2019,1395.00
1,Alexander Nicoll,2020,1705.10
2,Alexander Nicoll,2021,1739.50
3,Alexander Nicoll,2022,1764.25
4,Alexander Nicoll,2023,1531.10


In [107]:
## PPG Per Year
dfPPGYearbyYear = dfTeamHistory[dfTeamHistory['Type'] != 'Playoff'].groupby(['Team', 'Year'])['Score'].mean().reset_index(name='PPG').sort_values(by=['Team', 'Year'], ascending=[True, True])
dfPPGYearbyYear.head()

,Team,Year,PPG
0,Alexander Nicoll,2019,116.250000
1,Alexander Nicoll,2020,131.161538
2,Alexander Nicoll,2021,124.250000
3,Alexander Nicoll,2022,117.616667
4,Alexander Nicoll,2023,139.190909


In [108]:
## Score Margin Per Year
dfScoreMarginYearbyYear = dfTeamHistory[dfTeamHistory['Type'] != 'Playoff'].groupby(['Team', 'Year'])['Score_margin'].mean().reset_index(name='Score Margin').sort_values(by=['Team', 'Year'], ascending=[True, True])
dfScoreMarginYearbyYear.head()

,Team,Year,Score Margin
0,Alexander Nicoll,2019,17.166667
1,Alexander Nicoll,2020,-13.276923
2,Alexander Nicoll,2021,-0.428571
3,Alexander Nicoll,2022,-7.430000
4,Alexander Nicoll,2023,7.509091


In [109]:
## Wins Per Year
dfWinsYearbyYear = dfTeamHistory[(dfTeamHistory['Outcome'] == 'Win') & (dfTeamHistory['Type'] != 'Playoff')].groupby(['Team', 'Year'])['Outcome'].count().reset_index(name='Wins').sort_values(by=['Team', 'Year'], ascending=[True, True])
dfWinsYearbyYear.head()

,Team,Year,Wins
0,Alexander Nicoll,2019,8
1,Alexander Nicoll,2020,5
2,Alexander Nicoll,2021,6
3,Alexander Nicoll,2022,5
4,Alexander Nicoll,2023,6


In [110]:
## Win Percentage Per Year
dfWinPctYearbyYear = dfTeamHistory[dfTeamHistory['Type'] != 'Playoff'].groupby(['Team', 'Year'])['Outcome'].apply(lambda x: (x == 'Win').mean() * 100).reset_index(name='Win %').sort_values(by=['Team', 'Year'], ascending=[True, True])
dfWinPctYearbyYear.head()

,Team,Year,Win %
0,Alexander Nicoll,2019,66.666667
1,Alexander Nicoll,2020,38.461538
2,Alexander Nicoll,2021,42.857143
3,Alexander Nicoll,2022,33.333333
4,Alexander Nicoll,2023,54.545455


In [111]:
## Join
dfYearbyYear = pd.merge(dfPointsYearbyYear, dfPPGYearbyYear, on=['Team', 'Year']).merge(dfScoreMarginYearbyYear, on=['Team', 'Year']).merge(dfWinsYearbyYear, on=['Team', 'Year']).merge(dfWinPctYearbyYear, on=['Team', 'Year'])
dfYearbyYear.head()

,Team,Year,Points,PPG,Score Margin,Wins,Win %
0,Alexander Nicoll,2019,1395.00,116.250000,17.166667,8,66.666667
1,Alexander Nicoll,2020,1705.10,131.161538,-13.276923,5,38.461538
2,Alexander Nicoll,2021,1739.50,124.250000,-0.428571,6,42.857143
3,Alexander Nicoll,2022,1764.25,117.616667,-7.430000,5,33.333333
4,Alexander Nicoll,2023,1531.10,139.190909,7.509091,6,54.545455


In [112]:
## Year Averages
dfPointsYearAvg = dfYearbyYear.sort_values(by='Year', ascending=True).groupby('Year')['Points'].mean().reset_index(name='Points')
dfPPGYearAvg = dfYearbyYear.sort_values(by='Year', ascending=True).groupby('Year')['PPG'].mean().reset_index(name='PPG')
dfWinsYearAvg = dfYearbyYear.sort_values(by='Year', ascending=True).groupby('Year')['Wins'].mean().reset_index(name='Wins')
dfYearbyYearAvg = pd.merge(dfPointsYearAvg, dfPPGYearAvg, on='Year').merge(dfWinsYearAvg, on='Year')
dfYearbyYearAvg['Score Margin'] = 0
dfYearbyYearAvg['Win %'] = 50

dfYearbyYearAvg.head()

,Year,Points,PPG,Wins,Score Margin,Win %
0,2017,1495.750,115.057692,6.375,0,50
1,2018,1552.375,129.364583,5.875,0,50
2,2019,1424.100,118.675000,5.900,0,50
3,2020,1789.020,137.616923,6.400,0,50
4,2021,1790.860,127.918571,7.000,0,50


### League Records

In [113]:
## Single Season Points (excluding playoffs)
dfTeamHistory[dfTeamHistory['Type'] != 'Playoff'].groupby(['Year', 'Team'])['Score'].sum().sort_values(ascending=False).reset_index(name='Total Season Score').head()

,Year,Team,Total Season Score
0,2022,Cody Morton,2061.0
1,2021,Gavin Tacto,1995.9
2,2020,Cody Morton,1971.3
3,2022,Gavin Tacto,1925.8
4,2022,Zachary Toma,1906.1


In [114]:
## Single Week Points (excluding playoffs)
dfTeamHistory[dfTeamHistory['Type'] != 'Playoff'].groupby(['Year', 'Week', 'Team'])['Score'].sum().sort_values(ascending=False).reset_index(name='Total Score').head()

,Year,Week,Team,Total Score
0,2020,3.0,Hayden Bingham,218.9
1,2020,1.0,Cody Morton,207.2
2,2018,10.0,Luca Hurst,202.0
3,2019,5.0,Luca Hurst,200.0
4,2021,2.0,Gavin Tacto,198.4


In [115]:
## Single Season Wins (excluding playoffs)
dfMostWinsEx = dfTeamHistory[dfTeamHistory['Type'] != 'Playoff'].groupby(['Year', 'Team'])['Outcome'].apply(lambda x: (x == 'Win').sum()).sort_values(ascending=False).reset_index(name='Total Wins')

# Merge on 'Year'
dfMostWinsEx = pd.merge(dfMostWinsEx, dfResults, on='Year')

# Create the 'Champion' column
dfMostWinsEx['Champion'] = np.where(dfMostWinsEx['Team'] == dfMostWinsEx['Champion'], 'yes', 'no')
# Create the 'Runner-Up' column
dfMostWinsEx['Runner-Up'] = np.where(dfMostWinsEx['Team'] == dfMostWinsEx['Runner-Up'], 'yes', 'no')
# Create the '3rd Place' column
dfMostWinsEx['3rd Place'] = np.where(dfMostWinsEx['Team'] == dfMostWinsEx['3rd Place'], 'yes', 'no')

dfMostWinsEx.sort_values(by='Total Wins', ascending=False).reset_index(drop=True).head()

,Year,Team,Total Wins,Champion,Runner-Up,3rd Place
0,2020,Dylan Peters,10,no,no,no
1,2021,Cody Morton,10,no,no,no
2,2018,Matt Willadsen,10,no,yes,no
3,2018,Luca Hurst,10,yes,no,no
4,2022,Hayden Bingham,9,no,no,no


In [116]:
## Single Season Wins (including playoffs)
dfMostWinsIn = dfTeamHistory.groupby(['Year', 'Team'])['Outcome'].apply(lambda x: (x == 'Win').sum()).sort_values(ascending=False).reset_index(name='Total Wins').head()

# Merge on 'Year'
dfMostWinsIn = pd.merge(dfMostWinsIn, dfResults, on='Year')

# Create the 'Champion' column
dfMostWinsIn['Champion'] = np.where(dfMostWinsIn['Team'] == dfMostWinsIn['Champion'], 'yes', 'no')
# Create the 'Runner-Up' column
dfMostWinsIn['Runner-Up'] = np.where(dfMostWinsIn['Team'] == dfMostWinsIn['Runner-Up'], 'yes', 'no')
# Create the '3rd Place' column
dfMostWinsIn['3rd Place'] = np.where(dfMostWinsIn['Team'] == dfMostWinsIn['3rd Place'], 'yes', 'no')

dfMostWinsIn.sort_values(by='Total Wins', ascending=False).reset_index(drop=True).head()

,Year,Team,Total Wins,Champion,Runner-Up,3rd Place
0,2018,Luca Hurst,12,yes,no,no
1,2018,Matt Willadsen,11,no,yes,no
2,2017,Dylan Peters,11,yes,no,no
3,2021,Gavin Tacto,11,yes,no,no
4,2022,Luca Hurst,11,yes,no,no


In [117]:
## Longest Win Streaks
dfStreaks = dfTeamHistory[['Year', 'Week', 'Team', 'Outcome']].sort_values(by=['Team', 'Year', 'Week']).reset_index(drop=True)
dfStreaks['Winning Streak'] = 0
dfStreaks['Losing Streak'] = 0

# Iterate over the rows and update the 'Winning Streak' column
for i in range(1, len(dfStreaks)):
    if (dfStreaks.loc[i, 'Team'] == dfStreaks.loc[i-1, 'Team']
        and dfStreaks.loc[i, 'Year'] == dfStreaks.loc[i-1, 'Year']
        and dfStreaks.loc[i, 'Outcome'] == 'Win'):
        dfStreaks.loc[i, 'Winning Streak'] = dfStreaks.loc[i-1, 'Winning Streak'] + 1
        dfStreaks.loc[i-1, 'Winning Streak'] = 0
    else:
        dfStreaks.loc[i, 'Winning Streak'] = 0

# Keep track of the maximum streak for each team
max_win_streaks = dfStreaks.groupby('Team')['Winning Streak'].max().reset_index(drop=True)

# Longest win streaks
dfStreaks[['Year', 'Team', 'Winning Streak']].sort_values(by='Winning Streak', ascending=False).reset_index(drop=True).head()

,Year,Team,Winning Streak
0,2018,Matt Willadsen,10
1,2019,Gavin Tacto,8
2,2018,Luca Hurst,7
3,2018,Nate Stone,5
4,2022,Luca Hurst,5


In [118]:
## Longest Lose Streaks

# Iterate over the rows and update the 'Losing Streak' column
for i in range(1, len(dfStreaks)):
    if (dfStreaks.loc[i, 'Team'] == dfStreaks.loc[i-1, 'Team']
        and dfStreaks.loc[i, 'Year'] == dfStreaks.loc[i-1, 'Year']
        and dfStreaks.loc[i, 'Outcome'] == 'Loss'):
        dfStreaks.loc[i, 'Losing Streak'] = dfStreaks.loc[i-1, 'Losing Streak'] + 1
        dfStreaks.loc[i-1, 'Losing Streak'] = 0
    else:
        dfStreaks.loc[i, 'Losing Streak'] = 0

# Keep track of the maximum streak for each team
max_loss_streaks = dfStreaks.groupby('Team')['Losing Streak'].max().reset_index(drop=True)

# Longest losing streaks
dfStreaks[['Year', 'Team', 'Losing Streak']].sort_values(by='Losing Streak', ascending=False).reset_index(drop=True).head()

,Year,Team,Losing Streak
0,2018,Zachary Toma,10
1,2017,Cody Morton,6
2,2022,Alexander Nicoll,6
3,2018,Cody Morton,6
4,2021,Luca Hurst,5


In [119]:
## Largest Wins
dfTeamHistory[dfTeamHistory['Type'] != 'Playoff'].sort_values(by='Score_margin', ascending=False).head()

,Year,Week,Team,Score,Opponent_score,Score_margin,Outcome,Type,Top_scoring_week,Lowest_scoring_week
448,2020,12.0,Dylan Peters,185.6,87.10,98.50,Win,Regular,1,0
242,2019,4.0,Dylan Peters,175.0,80.00,95.00,Win,Regular,1,0
785,2023,1.0,Alexander Nicoll,152.0,59.90,92.10,Win,Regular,0,0
418,2020,9.0,Cody Morton,195.8,105.30,90.50,Win,Regular,1,0
748,2022,13.0,Matt Willadsen,167.5,80.85,86.65,Win,Regular,1,0


In [120]:
dfMatchupsPd['Total Score'] = dfMatchupsPd['Score1'] + dfMatchupsPd['Score2']

## Lowest Scoring Matchups
dfLowScoringMatchupsAllTime = dfMatchupsPd.sort_values(by='Total Score', ascending=True)
dfLowScoringMatchupsAllTime.head()

## Highest Scoring Matchups
dfHighScoringMatchupsAllTime = dfMatchupsPd[dfMatchupsPd['Type'] != 'Playoff'][['Total Score', 'Year', 'Week', 'Team1', 'Score1', 'Team2', 'Score2']].sort_values(by='Total Score', ascending=False)
dfHighScoringMatchupsAllTime.head()

,Total Score,Year,Week,Team1,Score1,Team2,Score2
253,368.2,2020,3.0,Nate Stone,149.3,Hayden Bingham,218.9
146,362.0,2017,12.0,Dylan Peters,174.0,Gavin Tacto,188.0
123,353.0,2019,8.0,Gavin Tacto,177.0,Zachary Toma,176.0
412,346.0,2019,5.0,Hayden Bingham,146.0,Luca Hurst,200.0
176,344.9,2021,5.0,Cody Morton,183.6,Gavin Tacto,161.3


### Team Performance Summary

Team, Performance Metric League Rank, Performance Metric Value, Overall Player Performance

In [121]:
## Build Player Performance Data set
dfPlayerPerformance = pd.merge(dfPointsAllTime, dfPPGAllTime, on='Team').merge(dfPlayoffGamesAllTime, on='Team').merge(dfWinsAllTime, on='Team').merge(dfWinPctAllTime, on='Team')\
                          .merge(dfPlayoffWinsAllTime, on='Team').merge(dfTopScoringWeeksAllTime, on='Team').merge(dfLowScoringWeeksAllTime, on='Team').merge(dfScoreMarginAllTime, on='Team')

columns_to_rank = [
    'Points',
    'PPG',
    'Playoff Games',
    'Wins',
    'Win %',
    'Playoff Wins',
    'Top Scoring Weeks',
    'Low Scoring Weeks',
    'Avg Score Margin'
]

for column in columns_to_rank:
    rank_column = f'{column} Rank'
    ascending = False if column != 'Low Scoring Weeks' else True
    dfPlayerPerformance[rank_column] = dfPlayerPerformance[column].rank(method='min', ascending=ascending).astype(int)

## Overall Player Rank
dfPlayerPerformance['Average Rank'] = dfPlayerPerformance.filter(like='Rank').mean(axis=1)

dfPlayerPerformance = dfPlayerPerformance.sort_values(by='Average Rank', ascending=True).reset_index(drop=True)
dfPlayerPerformance.head()

,Team,Points,PPG,Playoff Games,Wins,Win %,Playoff Wins,Top Scoring Weeks,Low Scoring Weeks,Avg Score Margin,Points Rank,PPG Rank,Playoff Games Rank,Wins Rank,Win % Rank,Playoff Wins Rank,Top Scoring Weeks Rank,Low Scoring Weeks Rank,Avg Score Margin Rank,Average Rank
0,Luca Hurst,14219.25,129.925556,11,58,57.425743,8,18,10,3.436111,1,3,1,1,1,1,2,4,4,2.000000
1,Gavin Tacto,13781.25,131.981111,9,54,54.545455,5,20,7,5.568333,2,1,2,3,3,3,1,1,2,2.000000
2,Matt Willadsen,13293.15,127.305556,9,56,56.565657,6,15,13,3.499444,3,5,2,2,2,2,3,8,3,3.333333
3,Dylan Peters,12666.85,131.015000,4,51,54.255319,2,11,7,6.062778,4,2,6,4,4,5,5,1,1,3.555556
4,Hayden Bingham,12407.15,124.508333,6,45,46.875000,2,11,12,-2.296111,5,8,4,5,6,5,5,5,7,5.555556


In [122]:
## Top Peforming Metrics
df = dfPlayerPerformance.iloc[:, :-1]

top_performing_metrics = []

for team in df['Team'].unique():
  team_df = df[df['Team'] == team]
  metric_ranks = team_df.filter(like='Rank').T
  metric_ranks.columns = ['Rank']
  top_3_metrics = metric_ranks.nsmallest(3, 'Rank')
  top_3_metrics['Metric'] = top_3_metrics.index.str.replace(' Rank', '')
  top_3_metrics['Team'] = team
  top_3_metrics['League Rank'] = top_3_metrics['Rank']
  top_3_metrics['Value'] = team_df[top_3_metrics.index.str.replace(' Rank', '')].values[0].astype(int)
  top_performing_metrics.append(top_3_metrics[['Team', 'Metric', 'Value', 'League Rank']])

top_metrics_df = pd.concat(top_performing_metrics, ignore_index=True)

top_metrics_df['League Rank'] = top_metrics_df['League Rank'].apply(lambda x: f"{int(x)}{'th' if 10 <= int(x) <= 20 else {1: 'st', 2: 'nd', 3: 'rd'}.get(int(x) % 10, 'th')}")

top_dupes = top_metrics_df.duplicated(subset=['Metric', 'League Rank'], keep=False)
top_metrics_df.loc[top_dupes, 'League Rank'] = top_metrics_df.loc[top_dupes, 'League Rank'].apply(lambda x: 'Tied ' + x)
top_metrics_df.head()

,Team,Metric,Value,League Rank
0,Luca Hurst,Points,14219,1st
1,Luca Hurst,Playoff Games,11,1st
2,Luca Hurst,Wins,58,1st
3,Gavin Tacto,PPG,131,1st
4,Gavin Tacto,Top Scoring Weeks,20,1st


In [123]:
## Worst Performing Metrics
df = dfPlayerPerformance.iloc[:, :-1]

worst_performing_metrics = []

for team in df['Team'].unique():
  team_df = df[df['Team'] == team]
  metric_ranks = team_df.filter(like='Rank').T
  metric_ranks.columns = ['Rank']
  worst_3_metrics = metric_ranks.nlargest(3, 'Rank')
  worst_3_metrics['Metric'] = worst_3_metrics.index.str.replace(' Rank', '')
  worst_3_metrics['Team'] = team
  worst_3_metrics['League Rank'] = worst_3_metrics['Rank']
  worst_3_metrics['Value'] = team_df[worst_3_metrics.index.str.replace(' Rank', '')].values[0].astype(int)
  worst_performing_metrics.append(worst_3_metrics[['Team', 'Metric', 'Value', 'League Rank']])

worst_metrics_df = pd.concat(worst_performing_metrics, ignore_index=True)

worst_metrics_df['League Rank'] = worst_metrics_df['League Rank'].apply(lambda x: f"{int(x)}{'th' if 10 <= int(x) <= 20 else {1: 'st', 2: 'nd', 3: 'rd'}.get(int(x) % 10, 'th')}")

worst_dupes = worst_metrics_df.duplicated(subset=['Metric', 'League Rank'], keep=False)
worst_metrics_df.loc[worst_dupes, 'League Rank'] = worst_metrics_df.loc[worst_dupes, 'League Rank'].apply(lambda x: 'Tied ' + x)
worst_metrics_df.head()

,Team,Metric,Value,League Rank
0,Luca Hurst,Low Scoring Weeks,10,4th
1,Luca Hurst,Avg Score Margin,3,4th
2,Luca Hurst,PPG,129,3rd
3,Gavin Tacto,Wins,54,3rd
4,Gavin Tacto,Win %,54,3rd


## Table Write Outs

In [124]:
## Easiest/Toughest Opponent for each team
dfVersus.to_csv('/content/drive/My Drive/dfVersus.csv', index=False)

## Performance Summary for each team (Championships, Top 3 Finishes, Playoff Appearances)
historic_team_performance.to_csv('/content/drive/My Drive/historicteamperformance.csv', index=False)

## Year by Year Performance Summary for each team
dfYearbyYear.to_csv('/content/drive/My Drive/dfYearbyYear.csv', index=False)

## Year by Year Averages for entire league
dfYearbyYearAvg.to_csv('/content/drive/My Drive/dfYearbyYearAvg.csv', index=False)

## List the top 3 best performance metrics for each team
top_metrics_df.to_csv('/content/drive/My Drive/top_metrics_df.csv', index=False)

## Lists the top 3 worst performance metrics for each team
worst_metrics_df.to_csv('/content/drive/My Drive/worst_metrics_df.csv', index=False)